# Geospatial Analysis
Geospatial analysis is the gathering, display, and manipulation of imagery, GPS, satellite photography and historical data, described explicitly in terms of geographic coordinates or implicitly, in terms of a street address, postal code, or forest stand identifier as they are applied to geographic models. In this recipe, we will analyze such data from the city of Chicago. We will use Pandas to analyze this data.

In [1]:
import pandas as pd

## Landmark Data
The City of Chicago has a register of landmarks in the city, we can load in this data as a CSV:

In [4]:
landmarks = pd.read_csv('Individual_Landmarks_-_Map.csv')
landmarks['rowid'] = range(len(landmarks)) #unique number
landmarks

,LANDMARK NAME,ID,ADDRESS,DATE BUILT,ARCHITECT,LANDMARK DESIGNATION DATE,LATITUDE,LONGITUDE,LOCATION,rowid
0,Vassar Swiss Underwear Company Building,L-265,2543 - 2545 W Diversey Av,NaN,NaN,07/30/2008,41.931627,-87.692100,"(41.9316266084, -87.6921000957)",0
1,Mathilde Eliel House,L- 89,4122 S Ellis Av,1886,Adler & Sullivan,10/02/1991,41.819256,-87.602788,"(41.819255751, -87.6027879992)",1
2,Manhattan Building,L-139,431 S Dearborn St,1891,William LeBaron Jenney,07/07/1978,41.876066,-87.628964,"(41.8760657234, -87.6289644505)",2
3,Machinery Hall at Illinois Institute of Techno...,L- 12,100 W 33rd St,1901,"Patton, Fisher & Miller",05/26/2004,41.835161,-87.629221,"(41.8351614122, -87.6292212235)",3
4,Melissa Ann Elam House,L- 88,4726 S Dr Martin Luther King Jr Dr,1903,Henry L. Newhouse,03/21/1979,41.808530,-87.617204,"(41.808529769, -87.6172043949)",4
...,...,...,...,...,...,...,...,...,...,...
312,(Former) Schlitz BreweryTied-House@1944 N.Oakley,L-310,1944 N. Oakley Ave.,1898,Kley & Lang,10/05/2011,41.917484,-87.685256,"(41.917484239, -87.6852556957)",312
313,Getty Tomb,L-103,"Graceland Cemetery, 4011 N Clark St",1890,Louis H. Sullivan,03/10/1971,41.961230,-87.661189,"(41.9612300749, -87.6611886208)",313
314,"Engine Company 129, Truck 50",L-236,8120 S Ashland Av,1928-29,Argyle E. Robinson,10/01/2003,41.746005,-87.663706,"(41.746005133, -87.6637057717)",314
315,James Charnley House,L- 65,1365 N Astor st,1891,"Adler and Sullivan, Frank Lloyd Wright",08/20/1972,41.907648,-87.627478,"(41.9076480171, -87.627477823)",315


The new type of data that we see in this dataset is latitude and longitude data. We use the library `geopy` to do manipulations of such numbers. For example, if we want to calculate the distance between landmarks, we can do the following:

In [6]:
import geopy.distance

def distance_between(i,j):
    coords_i = landmarks['LATITUDE'][i],landmarks['LONGITUDE'][i]
    coords_j = landmarks['LATITUDE'][j],landmarks['LONGITUDE'][j]
    
    return geopy.distance.geodesic(coords_i, coords_j).mi

distance_between(1,16)

5.515768144730867

Now, suppose, we would like to answer queries that identify nearby landmarks to you. So you provide the function your current latitude and longitude and it returns all landmarks within a distance of you. Naively, we could do the following:

In [8]:
import datetime

#take a coordinate for me
#find all landmarks within a distance of me
def find_naive(me, landmarks, distance=0.5):
    start = datetime.datetime.now()
    
    rtn = []
    
    N = len(landmarks)
    
    for i in range(N):
        coords_i = landmarks['LATITUDE'][i],landmarks['LONGITUDE'][i]
        
        if geopy.distance.geodesic(me, coords_i).mi < distance:
            rtn.append(i)
    
    print('Elapsed Time find_naive() ', (datetime.datetime.now()-start).total_seconds())
    
    return landmarks.loc[rtn]

#john crerar library: 41.790524,-87.6050427
find_naive((41.790524,-87.6050427), landmarks, distance=0.5)

Elapsed Time find_naive()  0.208709


,LANDMARK NAME,ID,ADDRESS,DATE BUILT,ARCHITECT,LANDMARK DESIGNATION DATE,LATITUDE,LONGITUDE,LOCATION,rowid
32,Frederick C. Robie House,L-176,5757 S Woodlawn Av,1909,Frank Lloyd Wright,09/15/1971,41.789920,-87.595970,"(41.7899203248, -87.5959702794)",32
73,Keck-Gottschalk-Keck Apartments,L-126,5551 S University Av,1937,George and William Keck,08/03/1994,41.793551,-87.597704,"(41.793550861, -87.5977036617)",73
202,Lorado Taft's Midway Studios,L-192,6016 S Ingleside Dr,1890-1929,"Arch ?, recon:1929;O F Johnson Add:1964;E D Dart",12/01/1993,41.785526,-87.603205,"(41.7855264091, -87.6032052025)",202
215,Site of the 1st Self-Sustain Cont. Nuclear Chain,L-184,East Side of S Ellis Ave between 56th & 57th St,1942,"Commern. sculpture,""Nuclear Energy"" Henry Moore",10/27/1971,41.792162,-87.600870,"(41.7921621211, -87.6008698951)",215
243,Rockefeller Memorial Chapel,L-178,1156-80 E 59th St,1925-28,Bertram Grosvenor Goodhue Associates,11/03/2004,41.788526,-87.597044,"(41.7885259092, -87.5970443072)",243
252,American School of Correspondence,L- 46,850 E 58th St,1907,Pond & Pond,04/15/1995,41.789754,-87.604104,"(41.789753922, -87.6041036284)",252


How do we make this search faster? Let's break the city up into sectors and do a sector-by-sector search.

To do so, let's create some dummy columns that bin the latitude and longitude into sectors.

In [9]:
landmarks['lat_bins'], latbins = pd.cut(x=landmarks['LATITUDE'], labels=False, bins=5, retbins=True)
landmarks['long_bins'], longbins = pd.cut(x=landmarks['LONGITUDE'], labels=False, bins=5, retbins=True)
landmarks[:10]

,LANDMARK NAME,ID,ADDRESS,DATE BUILT,ARCHITECT,LANDMARK DESIGNATION DATE,LATITUDE,LONGITUDE,LOCATION,rowid,lat_bins,long_bins
0,Vassar Swiss Underwear Company Building,L-265,2543 - 2545 W Diversey Av,NaN,NaN,07/30/2008,41.931627,-87.692100,"(41.9316266084, -87.6921000957)",0,3,2
1,Mathilde Eliel House,L- 89,4122 S Ellis Av,1886,Adler & Sullivan,10/02/1991,41.819256,-87.602788,"(41.819255751, -87.6027879992)",1,2,3
2,Manhattan Building,L-139,431 S Dearborn St,1891,William LeBaron Jenney,07/07/1978,41.876066,-87.628964,"(41.8760657234, -87.6289644505)",2,2,3
3,Machinery Hall at Illinois Institute of Techno...,L- 12,100 W 33rd St,1901,"Patton, Fisher & Miller",05/26/2004,41.835161,-87.629221,"(41.8351614122, -87.6292212235)",3,2,3
4,Melissa Ann Elam House,L- 88,4726 S Dr Martin Luther King Jr Dr,1903,Henry L. Newhouse,03/21/1979,41.808530,-87.617204,"(41.808529769, -87.6172043949)",4,2,3
5,(Former) Pioneer Trust and Savings Bank Building,L-318,4000 W. North Ave.,1924,Karl M. Vitzthum,06/06/2012,41.910192,-87.726617,"(41.9101921054, -87.7266173415)",5,3,1
6,DuPont-Whitehouse House,L- 85,3558 S Artesian Av,1876,Oscar Cobb & Co.,04/16/1996,41.828582,-87.686594,"(41.8285816489, -87.6865936818)",6,2,2
7,Montgomery Ward & Co. Catalog House,L-149,618 W Chicago Av,1907-08,"Richard E. Schmidt, Garden and Martin",05/17/2000,41.897437,-87.643695,"(41.8974368676, -87.6436954155)",7,3,3
8,Vorwaerts Turner Hall,L-286,2431 W. Roosevelt Rd,NaN,NaN,09/03/2009,41.866152,-87.687247,"(41.8661518103, -87.6872469444)",8,2,2
9,City Hall-County Building,L- 71,121 N LaSalle St / 118 N Clark St,1905-08,Holabird and Roche,01/21/1982,41.883843,-87.631655,"(41.8838425425, -87.6316552814)",9,3,3


We can now do something kind of cool, we can reindex this dataset by the defined bins so we can quickly pull up those landmarks inside the bins. This is how you build an inverted index in Pandas!

In [11]:
landmark_index = landmarks.groupby(['lat_bins','long_bins'])['rowid'].apply(list)
landmark_index

lat_bins  long_bins
0         2                                     [19, 116, 144, 221, 280]
          3                                                [12, 75, 118]
          4                            [11, 31, 100, 133, 188, 234, 311]
1         1                                                        [142]
          2                                 [35, 85, 183, 218, 232, 314]
          3            [26, 32, 45, 58, 73, 77, 79, 103, 132, 146, 17...
          4              [64, 67, 74, 165, 212, 228, 236, 264, 277, 289]
2         1                                          [34, 113, 159, 176]
          2            [6, 8, 10, 18, 27, 110, 127, 140, 199, 204, 21...
          3            [1, 2, 3, 4, 37, 39, 43, 60, 61, 65, 68, 72, 8...
3         0                            [71, 78, 102, 123, 181, 244, 316]
          1                                    [5, 42, 54, 93, 115, 167]
          2            [0, 16, 17, 22, 28, 47, 83, 87, 90, 98, 106, 1...
          3            [7, 9, 1

Now, we can write a more sophisticed find function that looks up only those elements in sector that contains your point.

In [12]:
def find_binned(me, landmarks, index, bins, distance=0.5):
    start = datetime.datetime.now()
    
    rtn = []
    
    #find the long and lat bin
    lat_bin_me = pd.cut(x=pd.Series(me[0]), labels=False, bins=bins[0])[0]
    long_bin_me = pd.cut(x=pd.Series(me[1]),labels=False, bins=bins[1])[0]
    
    for i in index[lat_bin_me, long_bin_me]:
        coords_i = landmarks['LATITUDE'][i],landmarks['LONGITUDE'][i]
        
        if geopy.distance.geodesic(me, coords_i).mi < distance:
            rtn.append(i)
    
    print('Elapsed Time find_binned() ', (datetime.datetime.now()-start).total_seconds())

    return landmarks.loc[rtn]

find_binned((41.790524,-87.6050427), landmarks, landmark_index, (latbins,longbins))

Elapsed Time find_binned()  0.036741


,LANDMARK NAME,ID,ADDRESS,DATE BUILT,ARCHITECT,LANDMARK DESIGNATION DATE,LATITUDE,LONGITUDE,LOCATION,rowid,lat_bins,long_bins
32,Frederick C. Robie House,L-176,5757 S Woodlawn Av,1909,Frank Lloyd Wright,09/15/1971,41.789920,-87.595970,"(41.7899203248, -87.5959702794)",32,1,3
73,Keck-Gottschalk-Keck Apartments,L-126,5551 S University Av,1937,George and William Keck,08/03/1994,41.793551,-87.597704,"(41.793550861, -87.5977036617)",73,1,3
202,Lorado Taft's Midway Studios,L-192,6016 S Ingleside Dr,1890-1929,"Arch ?, recon:1929;O F Johnson Add:1964;E D Dart",12/01/1993,41.785526,-87.603205,"(41.7855264091, -87.6032052025)",202,1,3
215,Site of the 1st Self-Sustain Cont. Nuclear Chain,L-184,East Side of S Ellis Ave between 56th & 57th St,1942,"Commern. sculpture,""Nuclear Energy"" Henry Moore",10/27/1971,41.792162,-87.600870,"(41.7921621211, -87.6008698951)",215,1,3
243,Rockefeller Memorial Chapel,L-178,1156-80 E 59th St,1925-28,Bertram Grosvenor Goodhue Associates,11/03/2004,41.788526,-87.597044,"(41.7885259092, -87.5970443072)",243,1,3
252,American School of Correspondence,L- 46,850 E 58th St,1907,Pond & Pond,04/15/1995,41.789754,-87.604104,"(41.789753922, -87.6041036284)",252,1,3


You get exactly the same results, a lot faster!! This technique works well if you are interested in searching for landmarks relatively close to you. The second that you cross bin boundaries it no longer works. Let's try to understand just how accurate binning chicago up into 25 sectors is. To do so, we need to compare the selected values with the binned algorithm with that of the naive algorithm.

We can do this calculation with a join. Luckily Pandas as a join feature called merge which allows us to find where two datasets intersect. In our case, we will use rowid to do this merge.

In [13]:
result_actual = find_naive((41.790524,-87.6050427), landmarks)
result_heuristic = find_binned((41.790524,-87.6050427), landmarks, landmark_index, (latbins,longbins))

common = result_actual.merge(result_heuristic,on='rowid')

print('Rows in result actual',len(result_actual))
print('Rows in result heuristic',len(result_heuristic))
print('Rows common to both', len(common) )


for dist in range(5,30,5):
    result_actual = find_naive((41.790524,-87.6050427), landmarks, distance=dist/10.0)
    result_heuristic = find_binned((41.790524,-87.6050427), landmarks, landmark_index, (latbins,longbins), distance=dist/10.0)
    common = result_actual.merge(result_heuristic,on='rowid')
    
    print('--- At Distance',dist/10.0,'---')
    tp = len(result_actual)
    print('Missing Rows',len(result_actual) - len(common))
    print()
    print()


Elapsed Time find_naive()  0.308589
Elapsed Time find_binned()  0.048922
Rows in result actual 6
Rows in result heuristic 6
Rows common to both 6
Elapsed Time find_naive()  0.25718
Elapsed Time find_binned()  0.017535
--- At Distance 0.5 ---
Missing Rows 0


Elapsed Time find_naive()  0.166732
Elapsed Time find_binned()  0.012499
--- At Distance 1.0 ---
Missing Rows 0


Elapsed Time find_naive()  0.196371
Elapsed Time find_binned()  0.01708
--- At Distance 1.5 ---
Missing Rows 7


Elapsed Time find_naive()  0.197826
Elapsed Time find_binned()  0.017398
--- At Distance 2.0 ---
Missing Rows 12


Elapsed Time find_naive()  0.219975
Elapsed Time find_binned()  0.031
--- At Distance 2.5 ---
Missing Rows 21


